In [1]:
import pandas as pd
import numpy as np
!pip install wget
import wget
import os

  Using cached wget-3.2-py3-none-any.whl


In [2]:
# download data
cc=wget.download("https://www.dropbox.com/s/dwa82x6xhjkhyw8/g3_FCM_distribution.feather?dl=1")

In [3]:
# this is some underway data collected from a cruise in 2019
g3 = pd.read_feather("../../../g3_FCM_distribution.feather")
print(g3.columns)
g3.head()

Index(['filename', 'pop', 'norm.scatter', 'norm.red', 'norm.orange',
       'norm.green', 'diam_lwr', 'Qc_lwr', 'diam_mid', 'Qc_mid', 'diam_upr',
       'Qc_upr', 'date', 'file', 'label', 'lat', 'lon', 'depth', 'replicate',
       'volume', 'stain', 'flag', 'comments'],
      dtype='object')


,filename,pop,norm.scatter,norm.red,norm.orange,norm.green,diam_lwr,Qc_lwr,diam_mid,Qc_mid,...,file,label,lat,lon,depth,replicate,volume,stain,flag,comments
0,towfish_001.fcs,prochloro,0.009565,0.008967,0.001417,0.003938,0.540813,0.030637,0.653386,0.049902,...,towfish_001,T20.28.05,30.0935,-158.0003,5,A,173,0,2,pro peak near low fluor noise
1,towfish_001.fcs,prochloro,0.008924,0.015568,0.001417,0.003938,0.532844,0.029486,0.643588,0.047994,...,towfish_001,T20.28.05,30.0935,-158.0003,5,A,173,0,2,pro peak near low fluor noise
2,towfish_001.fcs,prochloro,0.005311,0.011982,0.001417,0.003938,0.478654,0.022358,0.577122,0.036229,...,towfish_001,T20.28.05,30.0935,-158.0003,5,A,173,0,2,pro peak near low fluor noise
3,towfish_001.fcs,picoeuk,0.925120,1.580796,0.003468,0.003938,1.548094,0.462018,2.003069,0.898166,...,towfish_001,T20.28.05,30.0935,-158.0003,5,A,173,0,2,pro peak near low fluor noise
4,towfish_001.fcs,picoeuk,0.277725,0.207073,0.001417,0.003938,1.136778,0.208269,1.416857,0.367623,...,towfish_001,T20.28.05,30.0935,-158.0003,5,A,173,0,2,pro peak near low fluor noise


In [4]:
# get features out
feature_cols = ['norm.scatter', 'norm.red', 'norm.orange', 'diam_lwr', 'Qc_lwr', 'diam_mid', 
            'Qc_mid', 'diam_upr','Qc_upr', 'lat', 'lon', 'depth']
g3[feature_cols].describe()

,norm.scatter,norm.red,norm.orange,diam_lwr,Qc_lwr,diam_mid,Qc_mid,diam_upr,Qc_upr,lat,lon,depth
count,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1.744187e+06,1744187.0
mean,1.584134e-01,5.537662e-01,2.654904e-02,6.863884e-01,1.089720e-01,8.595890e-01,2.513584e-01,1.194107e+00,1.014238e+00,3.254085e+01,-1.580001e+02,5.0
std,5.742584e-01,1.792936e+00,8.778294e-02,4.059000e-01,3.371747e-01,6.113220e-01,1.128705e+00,1.184446e+00,5.610032e+00,4.790917e+00,2.211301e-03,0.0
min,8.902692e-04,2.227795e-03,4.386767e-04,3.324295e-01,8.729006e-03,3.988836e-01,1.396897e-02,4.955639e-01,2.445347e-02,2.424960e+01,-1.580264e+02,5.0
25%,4.767438e-03,8.630256e-03,4.787717e-04,4.680434e-01,2.110173e-02,5.641378e-01,3.416287e-02,7.053816e-01,6.080198e-02,2.883760e+01,-1.580004e+02,5.0
50%,9.147556e-03,1.708415e-02,6.348090e-04,5.357647e-01,2.990434e-02,6.471794e-01,4.868828e-02,8.124402e-01,8.754704e-02,3.345080e+01,-1.579999e+02,5.0
75%,4.089487e-02,3.257662e-01,3.046039e-03,7.357452e-01,6.778544e-02,8.959577e-01,1.126890e-01,1.142828e+00,2.111400e-01,3.678720e+01,-1.579995e+02,5.0
max,1.833011e+01,2.661446e+01,3.444115e+01,6.814193e+00,2.114438e+01,1.017273e+01,5.945244e+01,1.901778e+01,2.986843e+02,4.233380e+01,-1.579976e+02,5.0


In [5]:
# Labels are the values we want to predict
labels = np.array(g3['pop'])
labels = pd.get_dummies(labels)
# Remove the labels from the features
# axis 1 refers to the columns
features = g3[feature_cols]
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [6]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, 
                                                                            labels, 
                                                                            test_size=0.25,
                                                                            random_state=42)

In [7]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1308140, 12)
Training Labels Shape: (1308140, 3)
Testing Features Shape: (436047, 12)
Testing Labels Shape: (436047, 3)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
errors